In [ ]:
import gym
import time
import numpy as np

In [ ]:
ENV_NAME = 'CartPole-v0'
env = gym.make(ENV_NAME)

In [ ]:
env.reset()
for i in range(1000):
    env.render()
    env.step(env.action_space.sample())
    
# Had to add time.sleep in order to prevent python from crashing.    
    time.sleep(0.02)
    
    if i%10==0: print(i)
#    print(obs)
env.close()

In [ ]:
np.random.seed(1)
env.seed(1)

In [ ]:
hi_rew = 0
best_weights = None

In [ ]:
for i in range(200):
    obs = env.reset()
    weights = np.random.uniform(-1, 1, 4)
    sum_rew = 0
    for j in range(1000):
        env.render()
        action = 0 if np.matmul(weights, obs) < 0 else 1
        obs, rew, done, info = env.step(action)
        sum_rew += rew
        print(i, j, weights, obs, action, sum_rew, best_weights)
    if sum_rew > hi_rew:
        hi_rew = sum_rew
        best_weights = weights
    time.sleep(0.02)
env.close()

In [ ]:
print(hi_rew, best_weights)

In [ ]:
obs = env.reset()
for j in range(100):
    env.render()
    action = 0 if np.matmul(weights, obs) < 0 else 1
    obs, rew, done, info = env.step(action)
    print(j, action)
env.close()